## 1. Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

-   `numpy`: Used for numerical computations (matrix operations, random initialization, etc.).
-   `pandas`: Used to read the MNIST dataset from a CSV file.
-   `matplotlib.pyplot`: Used for visualizing handwritten digits from the dataset.

In [ ]:
# Load the dataset
# MNIST dataset should be stored in 'MNIST_Dataset.csv'
data = pd.read_csv('MNIST_Dataset.csv')

In [ ]:
# Convert pandas DataFrame to numpy array
data = np.array(data)
m, n = data.shape  # Get the number of rows (m) and columns (n)

In [ ]:
# Shuffle the dataset to ensure randomness before splitting
np.random.shuffle(data)

In [ ]:
# Split into development (dev) and training sets
# First 1000 examples for development/testing
data_dev = data[0:1000].T  # Transpose for better data handling
Y_dev = data_dev[0]  # Labels
X_dev = data_dev[1:n]  # Features
X_dev = X_dev / 255.  # Normalize pixel values (0-255) to (0-1)

In [ ]:
# Remaining data for training
data_train = data[1000:m].T  # Transpose
Y_train = data_train[0]  # Labels
X_train = data_train[1:n]  # Features
X_train = X_train / 255.  # Normalize
_, m_train = X_train.shape  # Get number of training examples

In [ ]:
# Initialize parameters (weights and biases) for a 2-layer neural network
def init_params():
    W1 = np.random.rand(10, 784) - 0.5  # Weights for layer 1
    b1 = np.random.rand(10, 1) - 0.5  # Biases for layer 1
    W2 = np.random.rand(10, 10) - 0.5  # Weights for layer 2
    b2 = np.random.rand(10, 1) - 0.5  # Biases for layer 2
    return W1, b1, W2, b2

In [ ]:
# Activation function: ReLU (Rectified Linear Unit)
def ReLU(Z):
    return np.maximum(Z, 0)

In [ ]:
# Activation function: Softmax (for multi-class classification)
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

In [ ]:
# Forward propagation: Computes activations through the network
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1  # Linear transformation for first layer
    A1 = ReLU(Z1)  # Activation using ReLU
    Z2 = W2.dot(A1) + b2  # Linear transformation for second layer
    A2 = softmax(Z2)  # Output probabilities using softmax
    return Z1, A1, Z2, A2

In [ ]:
# Derivative of ReLU function
def ReLU_deriv(Z):
    return Z > 0  # Returns 1 if Z > 0, otherwise 0

In [ ]:
# Convert labels into one-hot encoding
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y.T

In [ ]:
# Backpropagation: Compute gradients to update parameters
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)  # Convert labels to one-hot representation
    dZ2 = A2 - one_hot_Y  # Error at output layer
    dW2 = 1 / m_train * dZ2.dot(A1.T)  # Gradient for W2
    db2 = 1 / m_train * np.sum(dZ2, axis=1, keepdims=True)  # Gradient for b2
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)  # Error at hidden layer
    dW1 = 1 / m_train * dZ1.dot(X.T)  # Gradient for W1
    db1 = 1 / m_train * np.sum(dZ1, axis=1, keepdims=True)  # Gradient for b1
    return dW1, db1, dW2, db2

In [ ]:
# Update parameters using gradient descent
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [ ]:
# Predict the class label based on probabilities
def get_predictions(A2):
    return np.argmax(A2, axis=0)

In [ ]:
# Compute accuracy of the model
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [ ]:
# Training loop using gradient descent
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()  # Initialize weights and biases
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)  # Forward pass
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)  # Backward pass
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)  # Update parameters
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
# Function to make predictions
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    return get_predictions(A2)

In [ ]:
# Function to test predictions on a single example
def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]  # Select a single image
    prediction = make_predictions(current_image, W1, b1, W2, b2)  # Get prediction
    label = Y_train[index]  # Get actual label
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255  # Reshape to 28x28 pixels
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
# Train the model with learning rate 0.1 and 500 iterations
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

In [ ]:
# Test model predictions
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

In [ ]:
# Evaluate accuracy on development set
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
print("Dev Set Accuracy:", get_accuracy(dev_predictions, Y_dev))